# Preparação da base de perfil do eleitorado

[Voltar ao Índice](00_indice.ipynb)

Este notebook limpa e padroniza os dados de perfil do eleitorado.

In [1]:
import pandas as pd
import numpy as np
import utils as xu

## Hard-coded

In [2]:
simple_cols = ['PERIODO', 'UF', 'MUNICIPIO', 'COD_MUNICIPIO_TSE', 'NR_ZONA', 'SEXO', 'FAIXA_ETARIA', 
                   'GRAU_DE_ESCOLARIDADE', 'QTD_ELEITORES_NO_PERFIL']

compl_sel_cols  = ['ANO_ELEICAO', 'SG_UF', 'NM_MUNICIPIO', 'CD_MUNICIPIO', 'NR_ZONA', 'DS_GENERO', 
                   'DS_FAIXA_ETARIA', 'DS_GRAU_ESCOLARIDADE', 'QT_ELEITORES_PERFIL']

full_columns = ['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'CD_MUN_SIT_BIOMETRIA', 'DS_MUN_SIT_BIOMETRIA',
       'NR_ZONA', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'QT_ELEITORES_PERFIL',
       'QT_ELEITORES_BIOMETRIA', 'QT_ELEITORES_DEFICIENCIA',
       'QT_ELEITORES_INC_NM_SOCIAL']

## Funções

In [3]:
def load_simple_df(filename, simple_cols):

    df_raw = pd.read_csv(filename, sep=';', encoding='latin1', names=simple_cols)
    
    return df_raw

In [4]:
def simplify_dataset(compl_df, compl_sel_cols, simple_cols):
    """
    Given a "perfil do eleitorado" DataFrame `compl_df` that contains
    more info than the others (more columns), transform it to 
    the simpler version (same columns and column names).
    
    Input
    -----
    
    simple_cols : list of str
        Names of the columns of the simpler tables.
        
    compl_sel_cols : list of str
        Corresponding column names in the more comples tables.
    """
    
    # Cria dicionário que traduz nome das colunas:
    col_dict_compl = dict(zip(compl_sel_cols, simple_cols))
    # Cria tabela análoga as simples:
    simplified     = compl_df[compl_sel_cols].rename(col_dict_compl, axis=1)
    
    return simplified

In [5]:
def create_city_df(df):
    """
    Given a "perfil do eleitorado" DataFrame `df` (in the simpler setting, 
    with fewer columns), return a DataFrame of the UF and MUNICIPIO name, 
    with indices given by the TSE code.
    """
    
    # Get year of the dataset:
    years = df['PERIODO'].unique() if 'PERIODO' in df.columns else df['ANO_ELEICAO'].unique()
    assert len(years) == 1, 'There should be only one year in a dataset'
    year = years[0]

    # Extract list of municipalities:
    local_cols = ['COD_MUNICIPIO_TSE', 'UF', 'MUNICIPIO']
    locais     = df[local_cols].drop_duplicates()

    # Test that municipality code and name are biunivocal:
    locais['COMPLETO'] = locais['MUNICIPIO'] + ' (' + locais['UF'] + ')'
    test_biunivocal    = xu.one2oneQ(locais, 'COD_MUNICIPIO_TSE', 'COMPLETO')
    assert test_biunivocal, 'There is no one-to-one correspondence between place code and name.'

    locais     = locais.set_index('COD_MUNICIPIO_TSE')[['UF', 'MUNICIPIO']]
    new_cols   = [col + '_' + str(year) for col in locais.columns]
    col_mapper = dict(zip(locais.columns, new_cols))
    locais     = locais.rename(col_mapper, axis=1)

    return locais

In [6]:
def std_faixa_etaria(series):
    """
    Standardize column 'FAIXA_ETARIA' to more complex formatting 
    (e.g. lower case). Does not change the age ranges.
    """
    series = series.str.lower()
    series = series.str.replace('inválida', 'Inválido')
    series = series.apply(lambda s: s[0].upper() + s[1:])
    
    return series

In [7]:
def fix_sexo(series):
    series = series.str.replace('NAO INFORMADO', 'NÃO INFORMADO')
    return series

In [8]:
def fix_escolaridade(series):
    series = series.str.replace('NAO INFORMADO', 'NÃO INFORMADO')
    series = series.str.replace('LE E ESCREVE', 'LÊ E ESCREVE')
    #series = series.str.replace('PRIMEIRO GRAU', 'ENSINO FUNDAMENTAL')
    #series = series.str.replace('SEGUNDO GRAU', 'ENSINO MÉDIO')
    return series

In [9]:
def fix_old_data(df):
    df['SEXO'] = fix_sexo(df['SEXO'])
    df['GRAU_DE_ESCOLARIDADE'] = fix_escolaridade(df['GRAU_DE_ESCOLARIDADE'])
    return df

In [10]:
def fix_simple_data(df):
    df['FAIXA_ETARIA'] = std_faixa_etaria(df['FAIXA_ETARIA'])
    return df

In [11]:
def fix_complex_data(df):
    df['DS_FAIXA_ETARIA'] = df['DS_FAIXA_ETARIA'].str.strip()
    return df

In [12]:
def gen_compl_df(df, simple_cols, compl_sel_cols, full_columns):
    """
    Transform a simple eleitorado DataFrame `df` into a new DataFrame
    with the same columns as the complex versions. The columns are 
    empty.
    
    Input
    -----
    
    df : DataFrame
        The TSE "Perfil do eleitorado" data in the simpler form.
        
    simple_cols : list of str
        The names of the `df` columns.
        
    compl_sel_cols : list of str
        The new names of the `simple_cols` (the columns associated with them
        in the more complex tables)
        
    full_columns : list of str
        The names of all the columns in the complex TSE "Perfil do eleitorado" tables.
    """
    # Create structure:
    template = pd.DataFrame(data=None, columns=full_columns)
    # Rename common columns:
    col_dict_simple = dict(zip(simple_cols, compl_sel_cols))
    renamed = df.rename(col_dict_simple, axis=1)
    # Build DataFrame with all columns:
    result = pd.concat([template, renamed])
    
    return result

In [13]:
def get_cd_genero(ds_genero):
    """
    Translate gender description series `ds_genero` to a gender code series.
    """
    return ds_genero.map({'NÃO INFORMADO': 0, 'MASCULINO': 2, 'FEMININO': 4})

In [14]:
def get_cd_escolaridade(ds_escolaridade):
    """
    Translate academic record description series `ds_escolaridade` to a code series.
    """
    # Build description to code dict:
    ds_escolar = ['NÃO INFORMADO', 'ANALFABETO', 'LÊ E ESCREVE',
       'ENSINO FUNDAMENTAL INCOMPLETO', 'ENSINO FUNDAMENTAL COMPLETO',
       'ENSINO MÉDIO INCOMPLETO', 'ENSINO MÉDIO COMPLETO',
       'SUPERIOR INCOMPLETO', 'SUPERIOR COMPLETO', 'PRIMEIRO GRAU INCOMPLETO',
                 'PRIMEIRO GRAU COMPLETO', 'SEGUNDO GRAU INCOMPLETO', 'SEGUNDO GRAU COMPLETO']
    cd_escolar = [0, 1, 2, 3, 4, 5, 6, 7, 8, 3, 4, 5, 6]
    dict_escolar = dict(zip(ds_escolar, cd_escolar))
    
    return ds_escolaridade.map(dict_escolar)

In [15]:
def complete_df(df, simple_cols, compl_sel_cols, full_columns):
    """
    Transform a simple eleitorado DataFrame `df` into a new DataFrame
    with the same columns as the complex versions. The columns
    CD_GENERO and CD_GRAU_ESCOLARIDADE are filled based on their 
    description counterpart columns.
    
    Input
    -----
    
    df : DataFrame
        The TSE "Perfil do eleitorado" data in the simpler form.
        
    simple_cols : list of str
        The names of the `df` columns.
        
    compl_sel_cols : list of str
        The new names of the `simple_cols` (the columns associated with them
        in the more complex tables)
        
    full_columns : list of str
        The names of all the columns in the complex TSE "Perfil do eleitorado" tables.
    """

    # Create new columns and rename common ones:
    empty_complex = gen_compl_df(df, simple_cols, compl_sel_cols, full_columns)
    
    # Populate possible columns:
    empty_complex['CD_GENERO'] = get_cd_genero(empty_complex['DS_GENERO'])
    empty_complex['CD_GRAU_ESCOLARIDADE'] = get_cd_escolaridade(empty_complex['DS_GRAU_ESCOLARIDADE'])
    
    return empty_complex

## Constrói bases limpas e padronizadas

### 2022

In [16]:
eleitor2022 = pd.read_csv('/home/skems/ceweb/dados/brutos/tse/perfil_eleitorado_2022/perfil_eleitorado_2022.csv', sep=';', encoding='latin1')

In [17]:
eleitor2022 = fix_complex_data(eleitor2022)
use_2020_name = dict(zip(['CD_MUN_SIT_BIOMETRICA', 'DS_MUN_SIT_BIOMETRICA'], ['CD_MUN_SIT_BIOMETRIA', 'DS_MUN_SIT_BIOMETRIA']))
eleitor2022.rename(use_2020_name, axis=1, inplace=True)

In [18]:
#eleitor2022.sample(5).transpose()

In [21]:
#eleitor2022.to_csv('/home/skems/ceweb/dados/limpos/tse/perfil_eleitorado/perfil_eleitorado_2022.csv', index=False)

# Processo de construção e teste das transformações

Basicamente testes feitos no processo de construção da pipeline de limpeza acima.

## Carrega dados

In [24]:
eleitor2020 = pd.read_csv('../dados/limpos/perfil_eleitorado/perfil_eleitorado_2020.csv')

In [22]:
eleitor2022 = pd.read_csv('../dados/limpos/perfil_eleitorado/perfil_eleitorado_2022.csv')

#### Verifica consistência entre dados complexos

In [25]:
# Verifica que as colunas dos mais complexos são as mesmas:
(eleitor2022.columns == eleitor2020.columns).all()

True

In [28]:
# Verifica colunas que necessitam de um strip:
for c in eleitor2022.columns:
    if (eleitor2022[c].astype(str).str.strip() != eleitor2022[c].astype(str)).any():
        print(c)

DS_FAIXA_ETARIA
